In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import emlearn

In [ ]:
# Load datasets
train = pd.read_csv("dataset/train_radiation_data.csv")
test  = pd.read_csv("dataset/test_radiation_data.csv")

# Preprocessing function
def preprocess(df):
    """
    Preprocess the dataset with temporal parsing and encoding
    """
    df['time'] = pd.to_datetime(df['time'])
    df['hour'] = df['time'].dt.hour
    df['minute']  = df['time'].dt.minute
    df['weekofyr'] = df['time'].dt.isocalendar().week

    # Temporal encoding using sine and cosine
    df['hour_sin']  = np.sin(2 * np.pi * df['hour']/24)
    df['hour_cos']  = np.cos(2 * np.pi * df['hour']/24)
    df['min_sin']   = np.sin(2 * np.pi * df['minute']/60)
    df['min_cos']   = np.cos(2 * np.pi * df['minute']/60)
    df['woy_sin']   = np.sin(2 * np.pi * (df['weekofyr']-1)/52)
    df['woy_cos']   = np.cos(2 * np.pi * (df['weekofyr']-1)/52)
    return df

# Apply preprocessing
train = preprocess(train)
test  = preprocess(test)

# Features and Target
features = [
    'Gb(i)','Gd(i)','Gr(i)','H_sun','T2m','WS10m',
    'hour_sin','hour_cos','min_sin','min_cos','woy_sin','woy_cos'
]
target = ['P']

# Scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(train[features])
y_train_scaled = scaler_y.fit_transform(train[target])

X_test_scaled  = scaler_X.transform(test[features])
y_test_scaled  = scaler_y.transform(test[target])

# Sequence generation
LOOK_BACK = 16
HORIZON = 16

def create_sequences(X, y, look_back=LOOK_BACK, horizon=HORIZON):
    """
    Create sequences for time series prediction
    """
    Xs, ys = [], []
    for i in range(len(X) - look_back - horizon + 1):
        Xs.append(X[i:i + look_back])
        ys.append(y[i + look_back : i + look_back + horizon].flatten())
    return np.array(Xs), np.array(ys)

X_train, y_train = create_sequences(X_train_scaled, y_train_scaled)
X_test, y_test   = create_sequences(X_test_scaled, y_test_scaled)

# Flatten input for feed-forward model
n_features = len(features)
X_train_flat = X_train.reshape(X_train.shape[0], LOOK_BACK * n_features)
X_test_flat  = X_test.reshape(X_test.shape[0], LOOK_BACK * n_features)

# Build Feed-Forward Model
def build_ffnn_model(input_shape, horizon):
    """
    Modello feed-forward compatibile con emlearn
    """
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(input_shape,)))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(horizon, activation='linear'))

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Create and summarize model
model = build_ffnn_model(LOOK_BACK * n_features, HORIZON)
model.summary()

# Training
es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    X_train_flat, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    callbacks=[es],
    verbose=1
)


print(f"Final validation loss: {min(history.history['val_loss']):.6f}")
print(f"Final validation MAE: {min(history.history['val_mae']):.6f}")

# Evaluation
loss, mae = model.evaluate(X_test_flat, y_test)
print(f"\n\nTest Loss: {loss:.4f}, Test MAE: {mae:.4f}")

In [ ]:
print("Converting model for IoT deployment...")

cmodel = emlearn.convert(model, method='inline')
cmodel.save(file="../sensorPV/modelTiny.h", name='modelTiny')

In [ ]:
print("X min:", scaler_X.data_min_)
print("X max:", scaler_X.data_max_)
print("y min:", scaler_y.data_min_)
print("y max:", scaler_y.data_max_)